<a href="https://colab.research.google.com/github/daniel-mckemie/data71200_Su24_McKemie/blob/main/DMcKemie_DATA71200_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
import numpy as np
import pandas as pd

!pip install -U scikit-learn==1.4

In [179]:
import os
import tarfile
from six.moves import urllib
import io
import pandas as pd

GGITHUB_REPO_URL = "https://raw.githubusercontent.com/daniel-mckemie/data71200_Su24_McKemie/main/"
TAR_FILE_NAME = "tmb_mskcc_2018.tar"
DATA_FILE_NAME_PATIENT = "tmb_mskcc_2018/data_clinical_patient.txt"
DATA_FILE_NAME_SAMPLE = "tmb_mskcc_2018/data_clinical_sample.txt"
DATA_FILE_NAME_GENE_PANEL_MATRIX = "tmb_mskcc_2018/data_gene_panel_matrix.txt"
DATA_FILE_NAME_SV = "tmb_mskcc_2018/data_sv.txt"
DATA_FILE_NAME_MUTATIONS = "tmb_mskcc_2018/data_mutations.txt"

WITHOUT_MUTATION_DATA = "without_mutation_data.csv"
WITH_MUTATION_DATA = "with_mutation_data.csv"

DATA_FILES = [
    DATA_FILE_NAME_PATIENT,
    DATA_FILE_NAME_SAMPLE,
    DATA_FILE_NAME_GENE_PANEL_MATRIX,
    DATA_FILE_NAME_SV,
    DATA_FILE_NAME_MUTATIONS,

]


In [180]:
def load_data_from_github(github_url=GITHUB_REPO_URL, tar_file=TAR_FILE_NAME, data_files=DATA_FILES):
    tar_url = github_url + tar_file

    # Download the tar file to a local temporary file
    temp_file, _ = urllib.request.urlretrieve(tar_url)

    try:
        dataframes = {}
        # Open the tar file from the local temporary file
        with tarfile.open(temp_file, mode="r") as data_tar:
            for data_file in data_files:
                # Extract the specific txt file (now with correct path)
                extracted_file = data_tar.extractfile(data_file)
                if extracted_file:
                    # Read the content of the extracted file
                    file_content = extracted_file.read()
                    # Decode the content (assuming it's encoded in 'utf-8')
                    decoded_content = file_content.decode('utf-8')
                    # Create a StringIO object from the decoded content
                    string_io = io.StringIO(decoded_content)
                    df = pd.read_csv(string_io, sep=None, engine='python', skiprows=0)
                    if data_file in [DATA_FILE_NAME_PATIENT, DATA_FILE_NAME_SAMPLE]:
                        df = pd.read_csv(io.StringIO(decoded_content), sep=None, engine='python', skiprows=4)
                    dataframes[data_file] = df
                else:
                    print(f"File '{data_file}' not found in the tar archive.")
                    return None
        return dataframes
    finally:
        # Remove the temporary file to clean up
        os.remove(temp_file)


In [181]:
# QUESTION: What are correlations between mutation data and mortality and cancer type (possibly)?

# Load the data
dataframes = load_data_from_github()

# Check if dataframes were loaded
if dataframes:
    df_patient = dataframes.get(DATA_FILE_NAME_PATIENT)

    df_sample = dataframes.get(DATA_FILE_NAME_SAMPLE)

    df_gene_panel_matrix = dataframes.get(DATA_FILE_NAME_GENE_PANEL_MATRIX)

    df_sv = dataframes.get(DATA_FILE_NAME_SV)
    df_sv.columns = df_sv.columns.str.upper()

    df_mutations = dataframes.get(DATA_FILE_NAME_MUTATIONS)
    df_mutations.columns = df_mutations.columns.str.upper()




    merged_df = pd.merge(df_patient, df_sample, on="PATIENT_ID")

    merged_df = pd.merge(merged_df, df_gene_panel_matrix, on="SAMPLE_ID")

    # Only keep patients who have gene/sample info
    merged_df = pd.merge(merged_df, df_sv, on="SAMPLE_ID", how="right")

    # A few dozen duplicates, arguably not needed.  Prune...
    merged_df.drop_duplicates(subset=['PATIENT_ID'], inplace=True)

    # Change variable name for clarity
    without_mutation_df = merged_df

    print('Success')


else:
    print("Cannot merge dataframes as one or both are missing.")



Success


In [182]:
# Save the merged DataFrame to a CSV file
without_mutation_df.to_csv(WITHOUT_MUTATION_DATA, index=False)

# Full data for perhaps later use
with_mutation_df = pd.merge(merged_df, df_mutations, left_on="SAMPLE_ID", right_on="TUMOR_SAMPLE_BARCODE", how="left")
with_mutation_df.to_csv(WITH_MUTATION_DATA, index=False)

In [183]:
y = without_mutation_df['OS_STATUS']

In [188]:
# Drop useless columns
X = without_mutation_df.drop(['SITE2_EFFECT_ON_FRAME'], axis=1)